In [ ]:
import numpy as np                         # np is the alias we can now use to access any tool in numpy
import math
from matplotlib import pyplot as plt       # here, we only need the pyplot module from the library (with alias plt)
import pandas as pd
import scipy
from scipy import special                  # import the special functions, such as "gamma functions"
import scipy.optimize as opt               # this is needed for chi-squared minimization
from scipy.optimize import curve_fit       # this is the curve-fitting function
from itertools import product

# Edit This
This is the part that should be edited after the code meets basic standards to match the situation (i.e. alter the size of the room, etc.)

In [ ]:
cube_size = 1000 #Side length of the cube
chunk_number = 5 #The number of chunks along each axis

# Actual Code

In [ ]:
elements_per_chunk = int(cube_size / chunk_number)
x_semi_range = range(0, elements_per_chunk, 1) #The range of x values in a given chunk
y_semi_range = range(0, elements_per_chunk, 1)
z_semi_range = range(0, elements_per_chunk, 1)

i_range = range(0,chunk_number,1) #Define the chunks
j_range = range(0,chunk_number,1)
k_range = range(0,chunk_number,1)

room = np.zeros(shape=(cube_size,cube_size,cube_size)) #this creates a virtual room that is filled with zeros (representing zero detected sources at each point).
non_chunks_matrix = np.zeros(shape=(chunk_number, chunk_number, chunk_number)) #This matrix is greater than 0 in any chunk where we know there is no source

In [ ]:
def find_chunk(position):
  """
    This function will determine which chunk the robot is in
  """
  i = np.floor(position[0] / elements_per_chunk)
  j = np.floor(position[1] / elements_per_chunk)
  k = np.floor(position[2] / elements_per_chunk)

  return [i,j,k]

def find_non_chunks(chunk_position, x_sign, y_sign, z_sign):
  new_non_chunks_matrix = np.zeros(shape=(chunk_number, chunk_number, chunk_number))

  i, j, k = chunk_position
  i = np.int(i)
  j = np.int(j)
  k = np.int(k)

  if chunk_position[0] > 0:
    if x_sign > 0:
      new_non_chunks_matrix[i-1,j,k] += 1
  if chunk_position[0] < chunk_number-1:
    if x_sign < 0:
      new_non_chunks_matrix[i+1,j,k] += 1

  if chunk_position[1] > 0:
    if y_sign > 0:
      new_non_chunks_matrix[i,j-1,k] += 1
  if chunk_position[1] < chunk_number-1:
    if y_sign < 0:
      new_non_chunks_matrix[i,j+1,k] += 1

  if chunk_position[2] > 0:
    if z_sign > 0:
      new_non_chunks_matrix[i,j,k-1] += 1
  if chunk_position[2] < chunk_number-1:
    if z_sign < 0:
      new_non_chunks_matrix[i,j,k+1] += 1

  return new_non_chunks_matrix


def create_lines(position, angle, room_state, non_chunks_matrix):
  """
  This function will return the room matrix with 1s in all of the possible locations for radiation sources
    (i.e. all of the spaces where a radiation source could be to give the angle at the position)

  Inputs:
    position - a vector containing the x, y, and z position of the robot
    angle - a vector containing the mu and gamma for the angle from the robot to the source (with straight ahead being <0,pi/2> and going counter-clockwise)
    room_state - the 3d room matrix, which is updated and returned.
    no_chunks_matrix - a 3d matrix of the chunks with non-zero values for chunks that are confirmed to not have sources.
  """
  mu = angle[0]
  gamma = angle[1]

  step_x = np.sqrt(1 - mu ** 2) * np.cos(gamma) #defines the x component of the unit vector
  step_y = np.sqrt(1 - mu ** 2) * np.sin(gamma) #defines the y component of the unit vector
  step_z = mu                                   #defines the z component of the unit vector

  which_chunk = find_chunk(position)
  new_non_chunks_matrix = non_chunks_matrix + find_non_chunks(which_chunk, np.sign(step_x), np.sign(step_y), np.sign(step_z))

  x_coordinate = position[0]
  y_coordinate = position[1]
  z_coordinate = position[2]

  in_bounds = True #
  while in_bounds:
    if np.abs(x_coordinate - (cube_size / 2)) > (cube_size / 2) - 3:
      in_bounds = False
    if np.abs(y_coordinate - (cube_size / 2)) > (cube_size / 2) - 3:
      in_bounds = False
    if np.abs(z_coordinate - (cube_size / 2)) > (cube_size / 2) - 3:
      in_bounds = False

    x_coordinate += step_x
    y_coordinate += step_y
    z_coordinate += step_z

    room_state[math.ceil(x_coordinate)][math.ceil(y_coordinate)][math.ceil(z_coordinate)] += 1

  return room_state, new_non_chunks_matrix

def get_angle(robot_location, source_location):
  dx = source_location[0] - robot_location[0]
  dy = source_location[1] - robot_location[1]
  dz = source_location[2] - robot_location[2]
  r = np.sqrt(dx ** 2 + dy ** 2 + dz ** 2)

  mu = dz / r
  if dx != 0:
    theta = np.arctan(dy/dx)
  else:
    theta = np.pi / 2

  return [mu, theta]

In [ ]:
source1 = [500,550,600]

robot_location1 = [500,500,5]
robot_location2 = [5,900,10]
robot_location3 = [500,10,10]
robot_location4 = [400,400,400]

room, non_chunks_matrix = create_lines(robot_location1, get_angle(robot_location1, source1), room, non_chunks_matrix)
room, non_chunks_matrix = create_lines(robot_location2, get_angle(robot_location2, source1), room, non_chunks_matrix)
room, non_chunks_matrix = create_lines(robot_location3, get_angle(robot_location3, source1), room, non_chunks_matrix)
room, non_chunks_matrix = create_lines(robot_location3, get_angle(robot_location4, source1), room, non_chunks_matrix)

<ipython-input-4-a1ea30ff2e1c>:15: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  i = np.int(i)
<ipython-input-4-a1ea30ff2e1c>:16: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guida

In [ ]:
max_value = 0
max_position = [-1, -1, -1]

for i in i_range: #Iterate across the chunks
  for j in j_range:
    for k in k_range:
      if non_chunks_matrix[i, j, k] == 0:
        for x_semi in x_semi_range: #Iterate within the chunks
          for y_semi in y_semi_range:
            for z_semi in z_semi_range:
              x_abs = (i * elements_per_chunk) + x_semi
              y_abs = (j * elements_per_chunk) + y_semi
              z_abs = (k * elements_per_chunk) + z_semi

              if room[x_abs][y_abs][z_abs] > max_value:
                max_value = room[x_abs][y_abs][z_abs]
                max_position = [x_abs, y_abs, z_abs]
      else:
        print(i, j, k)

1 0 0
1 2 0
2 1 0


In [ ]:
print(max_value, max_position)
print([max_position[0] - source1[0], max_position[1] - source1[1], max_position[2] - source1[2]])

3.0 [500, 551, 601]
[0, 1, 1]
